<a href="https://colab.research.google.com/github/renzoziegler/data_engineering_bootcamp/blob/main/Bootcamp_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Temos duas bases de dados pessoais (fictícios)


*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada


Exercício 1:


*   Importar o Pandas
*   Instalar e importa a biblioteca fuzzywuzzy (será importante mais pra frente)
*   Ler os arquivos CSV fornecidos
*   Realizar uma análise exploratória dos dados



In [2]:
#Instalar libs
!pip install fuzzywuzzy

#Importar as libs necessárias
import pandas as pd
from fuzzywuzzy import fuzz

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#Ler os arquivos
cadastro = pd.read_csv('cadastro.csv')
dados_medicos = pd.read_csv('dados_medicos.csv')

Para análise exploratória, vamos entender as bases de dados: seu tamanho, quais dados estão disponíveis, qual o formato deles, se estão completos ou não.....

In [5]:
#Análise Exploratória
print(cadastro.shape) #393 regitros
print(dados_medicos.shape) #368 registros

(393, 7)
(368, 6)


In [6]:
print(cadastro.columns) #'nome', 'data_nasc', 'cpf', 'mae', 'celular', 'email', 'telefone_fixo'
print(dados_medicos.columns) #'nome', 'data_nasc', 'cpf', 'tipo_sanguineo', 'peso', 'altura'

Index(['nome', 'data_nasc', 'cpf', 'mae', 'celular', 'email', 'telefone_fixo'], dtype='object')
Index(['nome', 'data_nasc', 'cpf', 'tipo_sanguineo', 'peso', 'altura'], dtype='object')


In [19]:
print(cadastro.iloc[0])

nome                                           Cauã Isaac Caldeira
data_nasc                                      1945-03-02 00:00:00
cpf                                                    35083528118
mae                                         Vitória Kamilly Sueli 
celular                                                  989048340
email            cauaisaaccaldeira__cauaisaaccaldeira@hardquali...
telefone_fixo                                             28317237
nome_array                                 [Cauã, Isaac, Caldeira]
primeiro_nome                                                 Cauã
sobrenome                                                 Caldeira
nasc_ano                                                      1945
nasc_mes                                                         3
nasc_dia                                                         2
cpf_0                                                            3
Name: 0, dtype: object


In [20]:
print(dados_medicos.iloc[0])

nome                   Alice Liz Adriana Duarte
data_nasc                   1975-10-22 00:00:00
cpf                                 58548099313
tipo_sanguineo                               O-
peso                                         89
altura                                     1.57
nome_array        [Alice, Liz, Adriana, Duarte]
primeiro_nome                             Alice
sobrenome                                Duarte
nasc_ano                                   1975
nasc_mes                                     10
nasc_dia                                     22
cpf_0                                         5
Name: 0, dtype: object


In [21]:
print(cadastro.nome)

0                      Cauã Isaac Caldeira
1              Lúcia Carolina Luzia Almada
2              Vicente Marcos Edson Castro
3              Yago Pedro Henrique da Cruz
4          Valentina Stefany Alana Cardoso
                      ...                 
388    Arthur Luís Carlos Eduardo da Cunha
389                  Rayssa Simone Pereira
390                   Ester Emily Nogueira
391          Rodrigo Marcos Thomas Martins
392            Luna Esther Alessandra Dias
Name: nome, Length: 393, dtype: object


In [22]:
print(dados_medicos.nome)

0         Alice Liz Adriana Duarte
1                  Kaue César Lima
2      Otávio Felipe Henry Almeida
3         Rafaela Francisca Mendes
4       Bryan Joaquim Roberto Rosa
                  ...             
363     Jaqueline Vera Maya Moraes
364       Malu Emily Isabelly Lima
365        Rayssa D Priscila Alves
366           Emilly Sarah Drumond
367      Jorge Nelson Ricardo Mota
Name: nome, Length: 368, dtype: object


In [23]:
print(cadastro.dtypes)

nome                     object
data_nasc        datetime64[ns]
cpf                      object
mae                      object
celular                   int64
email                    object
telefone_fixo             int64
nome_array               object
primeiro_nome            object
sobrenome                object
nasc_ano                  int64
nasc_mes                  int64
nasc_dia                  int64
cpf_0                    object
dtype: object


In [24]:
print(dados_medicos.dtypes)

nome                      object
data_nasc         datetime64[ns]
cpf                       object
tipo_sanguineo            object
peso                       int64
altura                   float64
nome_array                object
primeiro_nome             object
sobrenome                 object
nasc_ano                   int64
nasc_mes                   int64
nasc_dia                   int64
cpf_0                     object
dtype: object


Temos no máximo 368 matches possíveis
E temos 144.624 possíveis combinações (393 x 368)

Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

In [10]:
#Pré-processamento
#Tamanhos dos campos
#Nome: Quebrar em nome e sobrenomes??
cadastro['nome_array'] = cadastro.nome.str.split()
cadastro['primeiro_nome'] = cadastro.nome_array.str[0]
cadastro['sobrenome'] = cadastro.nome_array.str[-1]

dados_medicos['nome_array'] = dados_medicos.nome.str.split()
dados_medicos['primeiro_nome'] = dados_medicos.nome_array.str[0]
dados_medicos['sobrenome'] = dados_medicos.nome_array.str[-1]

In [11]:
#CPF: Tem delimitador? está como tipo número ou tipo string? Tamanho 11 caracteres?
cadastro.cpf = cadastro.cpf.astype(str).apply(lambda x: x.zfill(11))
dados_medicos.cpf = dados_medicos.cpf.astype(str).apply(lambda x: x.zfill(11))

In [12]:
#Data Nascimento: Está como data?
cadastro.data_nasc = pd.to_datetime(cadastro.data_nasc, format = '%d-%m-%Y')
dados_medicos.data_nasc = pd.to_datetime(dados_medicos.data_nasc.str.replace('/','-'), format = '%d-%m-%Y')

In [13]:
cadastro['nasc_ano'] = cadastro.data_nasc.apply(lambda x : x.year)
cadastro['nasc_mes'] = cadastro.data_nasc.apply(lambda x : x.month)
cadastro['nasc_dia'] = cadastro.data_nasc.apply(lambda x : x.day)

dados_medicos['nasc_ano'] = dados_medicos.data_nasc.apply(lambda x : x.year)
dados_medicos['nasc_mes'] = dados_medicos.data_nasc.apply(lambda x : x.month)
dados_medicos['nasc_dia'] = dados_medicos.data_nasc.apply(lambda x : x.day)

In [14]:
#Altura como número?
dados_medicos.altura = dados_medicos.altura.str.replace(',','.').astype(float)

#Peso como número? OK

Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

In [15]:
#Indexação

#mês de nascimento
cadastro.groupby('nasc_mes').size()
dados_medicos.groupby('nasc_mes').size() #Muitas pessoas com nascimento em 1900
dados_medicos[dados_medicos['nasc_mes'] == 1]['data_nasc'].sort_values(ascending = False)

blocos = pd.concat([cadastro.groupby('nasc_mes').size(), dados_medicos.groupby('nasc_mes').size()], axis = 1)
blocos.columns = ['cadastro', 'dados_medicos']
print(sum(blocos.cadastro * blocos.dados_medicos)) #12.254 ao invés de 144.624

dados_medicos[dados_medicos['data_nasc'] == '1900-01-01']

cadastro['cpf_0'] = cadastro.cpf.str[0]
dados_medicos['cpf_0'] = dados_medicos.cpf.str[0]
cadastro.groupby('cpf_0').size()
dados_medicos.groupby('cpf_0').size()

blocos_cpf = pd.concat([cadastro.groupby('cpf_0').size(), dados_medicos.groupby('cpf_0').size()], axis = 1)
blocos_cpf.columns = ['cadastro', 'dados_medicos']
print(sum(blocos_cpf.cadastro * blocos_cpf.dados_medicos)) #14.862 ao invés de 144.624

cadastro_blocos = {}
dados_medicos_blocos = {}

for i in range(10):
    cadastro_blocos[i] = cadastro[cadastro['cpf_0'].astype(int) == i]
    dados_medicos_blocos[i] = dados_medicos[dados_medicos['cpf_0'].astype(int) == i]

12419
14726


Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

In [16]:
#Comparação par a par
def comparaRegistros(registro1, registro2):
    #Dictionary com similaridades
    sim = {}
    #Compara o CPF - exatamente igual
    if registro1['cpf'] == registro2['cpf']:
        sim['cpf'] = 1
    else:
        sim['cpf'] = 0
    
    #Compara Data de Nascimento - exatamente igual
    if registro1['data_nasc'] == registro2['data_nasc']:
        sim['data_nasc'] = 1
    elif (registro1['nasc_ano'] == registro2['nasc_ano']) and (registro1['nasc_mes'] == registro2['nasc_dia']) and (registro1['nasc_dia'] == registro2['nasc_mes']):
        #Mês e dia trocado!!
        sim['data_nasc'] = 1
    else:
        sim['data_nasc'] = 0

    #Compara Nomes - similaridade de strings:
    if registro1['nome'] == registro2['nome']:
        sim['nome'] = 1
    else:
        sim['nome'] = fuzz.token_sort_ratio(registro1['nome'], registro2['nome'])/100
        
    return sim

    #Outras possibilidades:
        #Usar nome da mãe na comparação
        #Comparar nome e sobrenome
        #Usar outro algoritmo de similaridade

In [17]:
pares = pd.DataFrame(columns = ['paciente', 'registro', 'similaridade'])

for i in range(0,10):
    print('CPF começando com {}'.format(i))
    #Varrer todos os dados_medicos, comparando com os registros em cadastro:
    for k, paciente in dados_medicos_blocos[i].iterrows():
        for j, registro in cadastro_blocos[i].iterrows():
            #print('Dados médicos: {}, {}, {}'.format(paciente.nome, paciente.cpf, paciente.data_nasc))
            #print('Cadastro: {}, {}, {}'.format(registro.nome, registro.cpf, registro.data_nasc))
            sim = comparaRegistros(paciente, registro)
            #print('Similaridade: {}, {}, {}'.format(sim['nome'], sim['cpf'], sim['data_nasc']))
            par = {'paciente' : paciente, 'registro' : registro, 'similaridade' : sim}
            pares = pares.append(par, ignore_index = True)

pares['simSum'] = 0

matches = pd.DataFrame()
matches_registros = pd.DataFrame()
potenciais = pd.DataFrame()
potenciais_registros = pd.DataFrame()
non_matches = pd.DataFrame()
non_matches_registros = pd.DataFrame()

CPF começando com 0
CPF começando com 1
CPF começando com 2
CPF começando com 3
CPF começando com 4
CPF começando com 5
CPF começando com 6
CPF começando com 7
CPF começando com 8
CPF começando com 9


Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

In [18]:
#Classificação
for idx, par in pares.iterrows():
    sim = par['similaridade']
    simSum = sim['cpf'] + sim['nome'] + sim['data_nasc']
    #Possibilidades - dar peso diferentes aos critérios
    pares.loc[idx, 'simSum'] = simSum
    if simSum == 3:
        matches = matches.append(pares.loc[idx])
        matches_registros = matches_registros.append(pares.loc[idx, 'registro'])
    elif simSum >=2:
        potenciais = potenciais.append(pares.loc[idx])
        potenciais_registros = potenciais_registros.append(pares.loc[idx, 'registro'])
    else:
        non_matches = non_matches.append(pares.loc[idx])
        non_matches_registros = non_matches_registros.append(pares.loc[idx, 'registro'])

is_missing = dados_medicos.merge(matches_registros, how = 'left', 
                on = ['nome', 'cpf', 'data_nasc'], indicator = True)
is_missing = is_missing[is_missing['_merge'] =='left_only'][['nome', 'data_nasc', 'cpf']]

Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [32]:
#Avaliação
%load_ext google.colab.data_table
potenciais[potenciais['simSum'] == 2].sample(10)
#True Positive: 10
#False Positive: 0

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,paciente,registro,simSum,similaridade
10701,nome Calebe Lucas Drumond dat...,nome Calebe Luc...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
7638,nome Arthur Paulo Enzo Duart...,nome Arthur P...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
3884,nome Nicole Alana Almada data...,nome Nicole Al...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
6085,nome Alícia Bárbara Cavalcant...,nome Alícia Bárbara ...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
14281,nome Ruan Theo Duarte data_na...,nome Ruan Theo Duarte dat...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
5237,nome Sarah Gabrielly Aragão d...,nome Sarah Gabrielly ...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
13090,nome Davi Elias Nascimento da...,nome Davi Elias Nascime...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
4523,nome Lúcia Stella Farias data...,nome Lúcia Ste...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
11637,nome Priscila Julia Márcia B...,nome Priscila J...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"
5551,nome Lucca Severino Martin R...,nome Lucca Sev...,2.0,"{'cpf': 1, 'data_nasc': 0, 'nome': 1}"


In [36]:
non_matches[non_matches['simSum']> 1.5].sample(10)
#True Negative: 2
#False Negative: 8

,paciente,registro,simSum,similaridade
4280,nome Anderson Raul Rafael As...,nome Anderson ...,1.98,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.98}"
9330,nome Enzo Albuquerque Silva d...,nome Antonio Albuquerque Silva...,1.85,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.85}"
11030,nome Sueli Clara Hadassa Cru...,nome Sueli Clara H...,1.94,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.94}"
4908,nome Alice Esther Rosa data_n...,nome ...,1.92,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.92}"
8274,nome Rodrigo Calebe João Gon...,nome Rodrigo C...,1.98,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.98}"
3557,nome Anthoni Raimundo Corte ...,nome Anthony Rai...,1.96,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.96}"
11225,nome Pedro Henrique Gabriel ...,nome Pedro Henri...,1.98,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.98}"
8730,nome Matheus Albuquerque Silv...,nome Antonio Albuquerque Silva...,1.80,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.8}"
2927,nome Gustavo Anthony Santos d...,nome Gustavo Ant...,1.92,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.92}"
13370,nome Calebe Gustavo Davi Cru...,nome Calebe Gustavo Davi da ...,1.94,"{'cpf': 1, 'data_nasc': 0, 'nome': 0.94}"


In [37]:
#True Positive: 10
#False Positive: 0
#True Negative: 2
#False Negative: 8
TP = 10
FP = 0
TN = 2
FN = 8

acuracia = (TP+TN)/(TP+FP+TN+FN)
precisao = TP/(TP+FP)
recall = TP/(TP+FN)
fmeasure = 2*(precisao*recall)/(precisao+recall)

print("Acuracia: {}".format(acuracia))
print("Recall: {}".format(recall))
print("Precisão: {}".format(precisao))
print("F-measure: {}".format(fmeasure))

Acuracia: 0.6
Recall: 0.5555555555555556
Precisão: 1.0
F-measure: 0.7142857142857143


Exercício 7: Revisão Manual (Clerical Review)
*    Com a classificação feita, que alterações nos valores de threshold poderíamos fazer, para minimizar os matches potenciais e melhorar a avaliação do modelo??